*Based on https://medium.com/vithelper/spatial-and-frequency-domain-image-processing-83ffa3fc7cbc https://www.cse.unr.edu/~bebis/CS485/*

# Image Filtering
## Methods
![title](img/spatial-to-frequency.png)
### Spatial Domain
$$
f(x, y) → \text{Operation }R → g(x, y)
$$
- *the spatial domain is represented as a 3D Vector of 2D matrices. Each 2D matrix contains the intensities for a single color.*
- Applying Operation R on image as spatial domain.
### Frequency Domain (i.e., Fourier Transform)
$$
f(x, y) → \text{Transform }T(u, v) → \text{Operation }R → \text{Inverse Transfrom} → g(x, y)
$$
- the methods which are based on Fourier Transform of an image.
- *Roughly, the term frequncy in an image tells about the rate of change of pixel values.*

![title](img/Screenshot_2668.png)
- [A real-time detection and positioning method for small and weak targets using a 1D morphology-based approach in 2D images](https://www.researchgate.net/publication/324947104_A_real-time_detection_and_positioning_method_for_small_and_weak_targets_using_a_1D_morphology-based_approach_in_2D_images)

### Point Processing Methods
    Converts a given pixel value to a new pixel value based on some predefined function.
![title](img/point_processing_methods.png)
- Negative
- Contrast stretching
- Thresholding
- Histogram Equalizaiton \
![title](img/hist_equal.png)

https://staff.fnwi.uva.nl/r.vandenboomgaard/IPCV20162017/LectureNotes/IP/PointOperators/HistogramEqualization.html

### Area Processing Methods
    Requires
    (1) Area shape and size -> Typically defined using a rectangular mask, **size is determined by mask size. size is an important parameter**
    (2) Operation -> Typically linear combination of pixel values. different results can be obtained using different weights. (e.g, smoothing, sharpening, edge detection etc...)
![title](img/area_processing_examples.png)

### Common Linear Operations
#### Corrleation
    A filtered image is generated as the **center** of the mask visits every pixel in the input image. (window-shifting)
![title](img/correlation.png)
##### Handling Pixels Close to Boundaries
- pad with zeros \
![title](img/padzero.png)
- wrap around (like a ball) \
![title](img/wrap.png)
##### Geometric Interporetation of Correlation
$$
\text{Suppose }x\text{ and }y\text{ are two n-dimensinal vectors: } \newline
x = (x_1,x_2, ..., x_n), y = (y_1, y_2, ..., y_n)
$$

$$
\text{The dot product of }x\text{ with }y\text{ is defined as: } \newline
x.y = x_1y_1 + x_2y_2 + ... + x_ny_n
$$

$$
\text{Using vector notation: } \newline
x.y = |x||y|cos(\theta)
$$

$$
cos(\theta)\text{ measures the similarity between }x\text{ and }y\text{: } \newline
x.y = |x||y|cos(\theta)\text{ or }cos(\theta) = \frac{xy}{|x||y|}
$$

$$
\text{Normalized correlation (i.e., divide by lengths): } \newline
$$
![title](img/normalized_correlation.png)

##### Normalized Correlation
    Measure the similarity between images or parts of images.
![title](img/normalized_correlation_example.png)
- Traditional correlation cannot handle changes
    - Size
    - Orientation
    - Shape (e.g., deformable objects)

#### Convolution
    Same as correlation except that the mask is flipped, both horizontally and veritcally.
    For symmertric masks (i.e., h(i,j) = h(-i, -j)), convolution is equivalent to correlation.
![title](img/correlation_vs_convolution.png)

#### How do we choose the mask weights?
    Usually by sampling certain functions and their derivatives.
![title](img/gaussian.png)

- Gaussian
    - Good for image smoothing
    - Good for sharpening

#### Normalization of Mask Weights
    Sum of weights affects overall intensity of output image
    - Property of derivative masks
    Positive weights - Normalize them such that they sum to one
    Positive/Negative Weights - Should sum to **zero** (But not always)
![title](img/mask_weights.png)

#### Smoothing Using Averaging
    Replace each pixel by the average of its neighbors.
    Useful for reducing noise and unimportant details.
    The size of the mask controls the amount of smoothing.
![title](img/smoothing.png)

- Trade-off : noise vs blurring and loss of detail.
![title](img/smoothing-examples.png)

#### Gaussian Smoothing
    Replace each pixel by a weighted average of its neighbors
    Mask weights are computed by sampling a Gaussian function
![title](img/gaussian_smoothing.png)
mask size depends on $\sigma \text{: height = width = }5\sigma \text{ (subtends 98.76\% of the area)}$ \
$\sigma$ determines the degree of smoothing!
![title](img/gaussian_smoothing_sigma.png)

##### Properties of Gaussian
- Convolution with self is anothor Gaussian \
$G_{\sigma_1}(X) \cdot G_{\sigma_2}(X)=G_{\sqrt{\sigma_1^2+\sigma_2^2}}(X)$
- Convolving two times with Gaussian kernel of width $\sigma$ is equivalent to convolving once with kernel of width $\sigma{\sqrt{2}}$
- Separable kernel
    - a 2D Gaussian can be expressed as the product of two 1D Gaussians.
    $$G_\sigma(x,y) = \frac{1}{2\pi\sigma^2}\exp({-\frac{x^2+y^2}{2\sigma^2})}
    = (\frac{1}{\sqrt{2{\pi}}{\sigma}}\exp({-\frac{x^2}{2\sigma^2}}) \cdot (\frac{1}{\sqrt{2{\pi}}{\sigma}}\exp({-\frac{y^2}{2\sigma^2}}))
    $$
    *OoC, interesting post : https://tex.stackexchange.com/questions/254785/e-vs-exp-in-display-mode*
- 2D Gaussian convolution can be implemented more efficiently using 1D convolutions
$$
g(i, j) = \sum_{k=-n/2}^{n/2}\sum_{l=-n/2}^{n/2}h(k,l)f(i-k, j-l) = \newline
\sum_{k=-n/2}^{n/2}\sum_{l=-n/2}^{n/2}\exp[\frac{-(k^2 + l^2)}{2\sigma^2}]f(i-k, j-l) = \newline
\sum_{k=-n/2}^{n/2}\exp[\frac{-k^2}{2\sigma^2}]\sum_{l=-n/2}^{n/2}\exp[\frac{-l^2}{2\sigma^2}]f(i-k, j-l)
$$
- Example \
    ![title](img/gaussian_2d.jpg)\
    $O(n^2)$
    ![title](img/1d_gaussian.png)\
    $O(2n) = O(n)$

- Process
    - To convolve an image $I$ with a $n \times n$ 2D Gaussian mask $G$ with $\sigma = \sigma_g$
        1. Build a 1D Gaussian mask $g$, of width $n$, with $\sigma_g$
        2. Convolve each roch of $I$ with $g$, get a new image $I_r$
        3. Convolve each column of $I_r$ with $g$

#### Image Sharpening
    Compute intensity differences in local image regions.
    Useful for emphasizing transitions in intensity (e.g., in edge detection)
![title](img/gaussian_sharpening.png)